In [8]:
import pandas as pd
import plotly.graph_objects as go

 
# csv_url = "https://your-bucket.s3.amazonaws.com/map_data.csv" 
csv_url = "map_data.csv" 

# 1. CSV 로딩
df = pd.read_csv(csv_url)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# 2. 시각화용 값 추출
names = df['name'].unique()
timestamps = sorted(df['timestamp'].unique())

fig = go.Figure()
trace_info = []

# 3. 라인/폴리곤 trace 생성
for name_idx, name in enumerate(names):
    for ts in timestamps:
        filtered = df[(df['name'] == name) & (df['timestamp'] == ts)]
        if filtered.empty:
            continue

        trace_type = filtered['type'].iloc[0]
        visible = (name_idx == 0 and ts == timestamps[0])
        date_label = pd.to_datetime(ts).date()
        label_prefix = "[POLYGON]" if trace_type == "polygon" else "[LINE]"
        color = 'blue' if trace_type == "polygon" else 'red'

        fig.add_trace(go.Scattermapbox(
            mode="lines",
            lon=filtered["lon"],
            lat=filtered["lat"],
            fill="toself" if trace_type == "polygon" else None,
            line=dict(width=3, color=color),
            name=f"{label_prefix} {name} @ {date_label}",
            visible=visible
        ))
        trace_info.append((name, ts, trace_type))

# 4. 슬라이더 생성
slider_steps = []
for ts in timestamps:
    step = {
        "method": "update",
        "label": ts.strftime('%Y-%m-%d'),
        "args": [{"visible": [False] * len(fig.data)}]
    }
    for i, (n, t, _) in enumerate(trace_info):
        if t == ts:
            step["args"][0]["visible"][i] = True
    slider_steps.append(step)

# 5. 드롭다운 생성
dropdown_buttons = []
for name in names:
    button = {
        "method": "update",
        "label": name,
        "args": [{"visible": [False]*len(fig.data)}]
    }
    for i, (n, _, _) in enumerate(trace_info):
        if n == name:
            button["args"][0]["visible"][i] = True
    dropdown_buttons.append(button)

# 6. 레이아웃 설정
fig.update_layout(
    sliders=[{
        "active": 0,
        "steps": slider_steps,
        "x": 0.1,
        "y": 0,
        "len": 0.9
    }],
    updatemenus=[{
        "buttons": dropdown_buttons,
        "direction": "down",
        "x": 0,
        "y": 1.1
    }],
    mapbox=dict(
        style="open-street-map",
        zoom=10,
        center={"lat": 37.5, "lon": 127.0}
    ),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    height=600
)

# 7. HTML로 저장
fig.write_html("interactive_map.html")


C:\Users\safom\AppData\Local\Temp\ipykernel_27112\2322380355.py:32: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

